### 参考链接：http://python.jobbole.com/87154/

In [3]:
# 通过yield创建生成器
def count(n):
    x = 0
    while x < n:
        yield x
        x += 1
for i in count(5):
    print(i)

0
1
2
3
4


In [9]:
# 通过迭代器创建序列
class CountIter:
    def __init__(self, n):
        self.n = n
    def __iter__(self):
        self.x = -1
        return self
    def __next__(self): # python2.x里面是next()
        self.x += 1
        if self.x < self.n:
            return self.x
        else:
            raise StopIteration

for i in CountIter(5):
    print(i)

0
1
2
3
4


In [11]:
# 使用生成器，每次next的时候，上下文都被保存下来
gen = count(2)
print(gen.__next__())
print(gen.__next__())
print(gen.__next__()) # 自动抛出StopIteration

0
1


StopIteration: 

In [13]:
# python提供的生成器表达式
gen = (x for x in range(5))
for i in gen:
    print(i)

gen1 = (x for x in range(5))
print(gen1.__next__())
print(gen1.__next__())

0
1
2
3
4
0
1


In [16]:
# close方法，可以关闭生成器。关闭后，就不能使用next()方法了。
gen = (x for x in range(5))
gen.close()
gen.__next__()

StopIteration: 

In [17]:
# send方法，向生成器传递参数
def count(n):
    x = 0
    while x < n:
        value = yield x
        if value is not None:
            print("recieved value: %s" % value)
        x += 1
gen = count(5)
print(gen.__next__())
# 字符串被传入生成器中，作为yield的关键字赋值给value
print(gen.send('hello'))

0
recieved value: hello
1


In [18]:
def throw_gen():
    try:
        yield 'Normal'
    except ValueError:
        yield 'Error'
    finally:
        print('Finally')

gen = throw_gen()
print(gen.__next__())
print(gen.__next__()) # 输出了finnaly并且抛出停止异常

Normal
Finally


StopIteration: 

In [22]:
def throw_gen():
    try:
        yield 'Normal'
    except ValueError:
        yield 'Error'
    finally:
        print('Finally')

gen = throw_gen()
print(gen.__next__())
print(gen.throw(ValueError)) #手动传递一个异常
print(gen.__next__())

Finally
Normal
Error
Finally


StopIteration: 

In [25]:
# 基于生成器的并发模拟
def consumer():
    last = ''
    while True:
        receival = yield last
        if receival is not None:
            print('Consume %s' % receival)
            last = receival
def producer(gen, n):
    gen.__next__() # 消费者准备开始执行
    x = 0
    while x < n:
        x += 1
        print('Produce %s' % x)
        last = gen.send(x) # 向消费者发送一个消费信号
    gen.close()
gen = consumer()
producer(gen, 5)

Produce 1
Consume 1
Produce 2
Consume 2
Produce 3
Consume 3
Produce 4
Consume 4
Produce 5
Consume 5
